In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import h5py
from v1dd_public import ARTIFACT_DIR
from allen_v1dd.client import OPhysClient

In [2]:
mouse_ids = ['M409828', 'M438833', 'M427836', 'M416296']
mouse_cres = ['slc2','slc4','slc5','teto1']

In [3]:
chase_h5_dir = ARTIFACT_DIR / "stimulus_analyses_v2.h5"
arefeh_mouse_csv_tag = "240610_lsn_ns_metrics"
merged_csv_tag = "240610"

In [4]:
# These are the directions for the drifting gratings stimulus. This is not the preferred way, but I used them for saving the preferred direction for DFG and DGW in the final CSV
directions = np.array([0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330])

### Merging the h5 with 4 CSV results
Note: I use the "is_roi_valid" from Chase's h5 to find the valid ROIs. In the final CSV there will be a column that has cell_index in each column, volume, and plane and a "valid_cell_index" which is when only valid cells are kept.
I also added a column to CSV "is_ignored_duplicate" that is indicating whether each cell is duplicate across planes or not. This is also from Chase's h5. This index is later used in my notebooks to exclude the duplicate cells. It is 1 for the first time the cell appears and 0 for all other duplicates.

This takes around 11 minutes

In [5]:
for mouse_num in range(len(mouse_ids)):
    mouse_id = mouse_ids[mouse_num]

    csv_path = ARTIFACT_DIR / f'{mouse_cres[mouse_num]}_{arefeh_mouse_csv_tag}.csv'
    csv_df = pd.read_csv(csv_path)
    merged_df = csv_df
    
    frac_resp_full_valid_all = []
    frac_resp_windowed_valid_all = []
    frac_resp_natural_images_valid_all = []
    frac_resp_natural_images_12_valid_all = []
    frac_resp_natural_movie_valid_all = []
    lsn_is_resp_valid_on_all = []
    lsn_is_resp_valid_off_all = []
    lsn_is_resp_valid_on_or_off_all = []
    valid_not_duplicate = []
    dgf_pref_dir_idx_valid_all = []
    dgw_pref_dir_idx_valid_all = []
    
    dgf_osi_valid_all = []
    dgw_osi_valid_all = []
    dgf_dsi_valid_all = []
    dgw_dsi_valid_all = []    
    dgf_ls_valid_all = []
    dgw_ls_valid_all = []
    dgf_runsp_valid_all = []
    dgw_runsp_valid_all = []    
    ns118_ls_valid_all = []
    ns12_ls_valid_all = []
    nm_ls_valid_all = []
    
    for index, row in csv_df.iterrows():
        column = str(row["column"])
        volume = str(row["volume"])
        col_vol_id = f"{column}{volume}"
        plane = row["plane"]
        valid_cell_index = row["valid_cell_index"]
        cell_index = row["cell_index"]
        
        with h5py.File(chase_h5_dir, 'r') as f:
            if f"{mouse_id}/{col_vol_id}/Plane_{plane}" in f:
                is_roi_valid = f[mouse_id][col_vol_id][f"Plane_{plane}"]["is_roi_valid"][()]
                frac_resp_full = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_full"]["frac_responsive_trials"]
                frac_resp_full_valid = frac_resp_full[is_roi_valid]
                
                not_duplicate = ~f[mouse_id][col_vol_id][f"Plane_{plane}"]["is_ignored_duplicate"][()]
                valid_indices = np.where(is_roi_valid)[0]
                valid_not_duplicate_all = not_duplicate[valid_indices]
                valid_not_duplicate.append(valid_not_duplicate_all[valid_cell_index])

                if is_roi_valid.shape[0] == frac_resp_full.shape[0]:
                    dgf_pref_dir_idx = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_full"]["pref_cond_index"][:]
                    dgf_pref_dir_idx_valid_cells = dgf_pref_dir_idx[is_roi_valid]
                    dgf_pref_dir_idx_valid = directions[dgf_pref_dir_idx_valid_cells[:,0]]
                    dgf_pref_dir_idx_valid_all.append(dgf_pref_dir_idx_valid.tolist()[valid_cell_index])
                    
                    dgf_osi = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_full"]["osi"]
                    dgf_osi_valid = dgf_osi[is_roi_valid]
                    dgf_osi_valid_all.append(dgf_osi_valid.tolist()[valid_cell_index])
                    
                    dgf_dsi = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_full"]["dsi"]
                    dgf_dsi_valid = dgf_dsi[is_roi_valid]
                    dgf_dsi_valid_all.append(dgf_dsi_valid.tolist()[valid_cell_index])

                    dgf_ls = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_full"]["lifetime_sparseness"]
                    dgf_ls_valid = dgf_ls[is_roi_valid]
                    dgf_ls_valid_all.append(dgf_ls_valid.tolist()[valid_cell_index])
                    
                    ns118_ls = f[mouse_id][col_vol_id][f"Plane_{plane}"]["natural_images"]["lifetime_sparseness"]
                    ns118_ls_valid = ns118_ls[is_roi_valid]
                    ns118_ls_valid_all.append(ns118_ls_valid.tolist()[valid_cell_index])
                    
                    ns12_ls = f[mouse_id][col_vol_id][f"Plane_{plane}"]["natural_images_12"]["lifetime_sparseness"]
                    ns12_ls_valid = ns12_ls[is_roi_valid]
                    ns12_ls_valid_all.append(ns12_ls_valid.tolist()[valid_cell_index])

                    nm_ls = f[mouse_id][col_vol_id][f"Plane_{plane}"]["natural_movie"]["lifetime_sparseness"]
                    nm_ls_valid = nm_ls[is_roi_valid]
                    nm_ls_valid_all.append(nm_ls_valid.tolist()[valid_cell_index])
                    
                    dgw_pref_dir_idx = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_windowed"]["pref_cond_index"][:]
                    dgw_pref_dir_idx_valid_cells = dgw_pref_dir_idx[is_roi_valid]
                    dgw_pref_dir_idx_valid = directions[dgw_pref_dir_idx_valid_cells[:,0]]
                    dgw_pref_dir_idx_valid_all.append(dgw_pref_dir_idx_valid.tolist()[valid_cell_index])

                    dgw_osi = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_windowed"]["osi"]
                    dgw_osi_valid = dgw_osi[is_roi_valid]
                    dgw_osi_valid_all.append(dgw_osi_valid.tolist()[valid_cell_index])
                    
                    dgw_dsi = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_windowed"]["dsi"]
                    dgw_dsi_valid = dgw_dsi[is_roi_valid]
                    dgw_dsi_valid_all.append(dgw_dsi_valid.tolist()[valid_cell_index])

                    dgw_ls = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_windowed"]["lifetime_sparseness"]
                    dgw_ls_valid = dgw_ls[is_roi_valid]
                    dgw_ls_valid_all.append(dgw_ls_valid.tolist()[valid_cell_index])
                    
                    if f"{mouse_id}/{col_vol_id}/Plane_{plane}/drifting_gratings_full/frac_responsive_trials" in f:
                        is_roi_valid = f[mouse_id][col_vol_id][f"Plane_{plane}"]["is_roi_valid"][()]
                        frac_resp_full = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_full"]["frac_responsive_trials"]
                        frac_resp_full_valid = frac_resp_full[is_roi_valid]
                        
                        frac_resp_full_valid_all.append(frac_resp_full_valid.tolist()[valid_cell_index])
                        
                    if f"{mouse_id}/{col_vol_id}/Plane_{plane}/drifting_gratings_windowed/frac_responsive_trials" in f:
                        frac_resp_windowed = f[mouse_id][col_vol_id][f"Plane_{plane}"]["drifting_gratings_windowed"]["frac_responsive_trials"]
                        frac_resp_windowed_valid = frac_resp_windowed[is_roi_valid]
                        frac_resp_windowed_valid_all.append(frac_resp_windowed_valid.tolist()[valid_cell_index])

                    else:
                        print(f"{mouse_id}/{col_vol_id}/Plane_{plane}/drifting_gratings_windowed/frac_responsive_trials did not exist in Chase's h5")

                    if f"{mouse_id}/{col_vol_id}/Plane_{plane}/natural_images/frac_responsive_trials" in f:                            
                        natural_images = f[mouse_id][col_vol_id][f"Plane_{plane}"]["natural_images"]["frac_responsive_trials"]
                        frac_resp_natural_images_valid = natural_images[is_roi_valid]
                        frac_resp_natural_images_valid_all.append(frac_resp_natural_images_valid.tolist()[valid_cell_index])

                    else:
                        print(f"{mouse_id}/{col_vol_id}/Plane_{plane}/natural_images/frac_responsive_trials did not exist in Chase's h5")
                    
                    if f"{mouse_id}/{col_vol_id}/Plane_{plane}/natural_images_12/frac_responsive_trials" in f:                                            
                        natural_images_12 = f[mouse_id][col_vol_id][f"Plane_{plane}"]["natural_images_12"]["frac_responsive_trials"]
                        frac_resp_natural_images_12_valid = natural_images_12[is_roi_valid]
                        frac_resp_natural_images_12_valid_all.append(frac_resp_natural_images_12_valid.tolist()[valid_cell_index])

                    else:
                        print(f"{mouse_id}/{col_vol_id}/Plane_{plane}/natural_images_12/frac_responsive_trials did not exist in Chase's h5")
                    
                    if f"{mouse_id}/{col_vol_id}/Plane_{plane}/natural_movie/frac_responsive_trials" in f:
                        natural_movie = f[mouse_id][col_vol_id][f"Plane_{plane}"]["natural_movie"]["frac_responsive_trials"]
                        frac_resp_natural_movie_valid = natural_movie[is_roi_valid]
                        frac_resp_natural_movie_valid_all.append(frac_resp_natural_movie_valid.tolist()[valid_cell_index])

                    else:
                        print(f"{mouse_id}/{col_vol_id}/Plane_{plane}/natural_movie/frac_responsive_trials did not exist in Chase's h5")

                    if f"{mouse_id}/{col_vol_id}/Plane_{plane}/locally_sparse_noise/is_responsive" in f:
                        lsn_resp = f[mouse_id][col_vol_id][f"Plane_{plane}"]["locally_sparse_noise"]["is_responsive"]
                        lsn_is_resp_valid_on = lsn_resp[is_roi_valid, 0]
                        lsn_is_resp_valid_on_all.append(lsn_is_resp_valid_on.tolist()[valid_cell_index])
                        lsn_is_resp_valid_off = lsn_resp[is_roi_valid, 1]
                        lsn_is_resp_valid_off_all.append(lsn_is_resp_valid_off.tolist()[valid_cell_index])
                        lsn_is_resp_valid_on_or_off = lsn_resp[is_roi_valid, 2]
                        lsn_is_resp_valid_on_or_off_all.append(lsn_is_resp_valid_on_or_off.tolist()[valid_cell_index])                        
                    else:
                        print(f"{mouse_id}/{col_vol_id}/Plane_{plane}/locally_sparse_noise/is_responsive did not exist in Chase's h5")
                                          
    merged_df["frac_resp_dgf"] = frac_resp_full_valid_all    
    merged_df["frac_resp_dgw"] = frac_resp_windowed_valid_all
    merged_df["frac_resp_ns118"] = frac_resp_natural_images_valid_all
    merged_df["frac_resp_ns12"] = frac_resp_natural_images_12_valid_all
    merged_df["frac_resp_nm"] = frac_resp_natural_movie_valid_all
    merged_df["is_responsive_on_z"] = lsn_is_resp_valid_on_all
    merged_df["is_responsive_off_z"] = lsn_is_resp_valid_off_all
    merged_df["is_responsive_on_or_off_z"] = lsn_is_resp_valid_on_or_off_all
    merged_df["valid_not_duplicate"] = valid_not_duplicate
    merged_df["dgf_pref_dir"] = dgf_pref_dir_idx_valid_all    
    merged_df["dgw_pref_dir"] = dgw_pref_dir_idx_valid_all
    merged_df["dgf_osi"] = dgf_osi_valid_all    
    merged_df["dgw_osi"] = dgw_osi_valid_all
    merged_df["dgf_dsi"] = dgf_dsi_valid_all    
    merged_df["dgw_dsi"] = dgw_dsi_valid_all
    merged_df["dgf_ls"] = dgf_ls_valid_all    
    merged_df["dgw_ls"] = dgw_ls_valid_all
    merged_df["ns12_ls"] = ns12_ls_valid_all    
    merged_df["ns118_ls"] = ns118_ls_valid_all
    merged_df["nm_ls"] = nm_ls_valid_all
    
    csv_path = ARTIFACT_DIR / f'{mouse_cres[mouse_num]}_all_stim_metrics_{merged_csv_tag}.csv'
    merged_df.to_csv(csv_path)

/tmp/ipykernel_4118847/37725589.py:5: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv(csv_path)
/tmp/ipykernel_4118847/37725589.py:5: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv(csv_path)
/tmp/ipykernel_4118847/37725589.py:5: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv(csv_path)
/tmp/ipykernel_4118847/37725589.py:5: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv(csv_path)
